In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans

from tqdm.notebook import tqdm 

from torch.utils.tensorboard import SummaryWriter

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.optim.lr_scheduler as lr_scheduler

from models.enhanced_baseline import EnhancedBaseLineModel
from ddpm.ddpm import GaussianDiffusion1D
from evaluation.evaluation import vizual_comparison, plot_jsd_per_customer, plot_kde_samples, make_gif_from_images, mmd_histogram_per_customer

# Params

In [2]:
## Data
seq_len = 48
batch_size = 128
k = 15

#NN
device = 'cuda' if torch.cuda.is_available() else 'cpu'
latent_dim = 1000
cond_model = "mlp"
num_layers = 6
n_heads = 8
lr = 5e-5
decay_rate = 0.9
epochs = 2000
save_rate = 100

## DDPM
timesteps = 1000
beta_schedule = "cosine"
objective = "pred_noise"

## Logging
experiment_name = "enhanced_dev"
logging_dir = f"./logging/{experiment_name}/"

In [3]:
if not os.path.isdir(logging_dir):
    os.makedirs(logging_dir)
    os.makedirs(os.path.join(logging_dir, "viz/"))
    os.makedirs(os.path.join(logging_dir, "jsd/"))
    os.makedirs(os.path.join(logging_dir, "kde/"))
    os.makedirs(os.path.join(logging_dir, "tensorboard/"))
    os.makedirs(os.path.join(logging_dir, "weights/"))
    os.makedirs(os.path.join(logging_dir, "ts_sample/"))
    os.makedirs(os.path.join(logging_dir, "mmd/"))

# Utils

In [4]:
PREPROCESSED_DIR = "./preprocessing/data/customer_led_network_revolution/preprocessed/"

In [5]:
class MakeDATA(Dataset):
    def __init__(self, data, seq_len):
        data = np.asarray(data, dtype=np.float32)
        seq_data = []
        for i in range(len(data) - seq_len + 1):
            x = data[i : i + seq_len]
            seq_data.append(x)
        self.samples = np.asarray(seq_data, dtype=np.float32) 

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

In [6]:
def cluster(data, k):
    kmeans = KMeans(n_clusters=k)
    cluster_labels = kmeans.fit_predict(data.T)
    
    clustered_data = []
    for cluster in range(kmeans.n_clusters):
        cluster_data = data.iloc[:, cluster_labels == cluster].mean(axis=1)
        clustered_data.append(cluster_data)
        
    return pd.DataFrame(clustered_data).T

# Load Data

In [7]:
train = pd.read_csv(os.path.join(PREPROCESSED_DIR, "train.csv"))
val = pd.read_csv(os.path.join(PREPROCESSED_DIR, "val.csv"))
test = pd.read_csv(os.path.join(PREPROCESSED_DIR, "test.csv"))
cond_train = pd.read_csv(os.path.join(PREPROCESSED_DIR, "cond_train.csv"))
cond_val = pd.read_csv(os.path.join(PREPROCESSED_DIR, "cond_val.csv"))
cond_test = pd.read_csv(os.path.join(PREPROCESSED_DIR, "cond_test.csv"))

In [8]:
train.drop("Date and Time of capture", axis=1, inplace=True)
val.drop("Date and Time of capture", axis=1, inplace=True)
test.drop("Date and Time of capture", axis=1, inplace=True)

In [9]:
train = cluster(train, k)
test = cluster(test, k)
val = cluster(val, k)

c:\Users\Arne\anaconda3\envs\hf_diff\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\Arne\anaconda3\envs\hf_diff\lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [10]:
train.shape, val.shape, test.shape, cond_train.shape, cond_val.shape, cond_test.shape

((33984, 15), (6796, 15), (1700, 15), (33984, 27), (6796, 27), (1700, 27))

In [11]:
train_seq = np.asarray(MakeDATA(train, seq_len)).transpose(0, 2, 1)
cond_train_seq = np.asarray(MakeDATA(cond_train, seq_len)).transpose(0, 2, 1)

val_seq = np.asarray(MakeDATA(val, seq_len)).transpose(0, 2, 1)
cond_val_seq = np.asarray(MakeDATA(cond_val, seq_len)).transpose(0, 2, 1)

test_seq = np.asarray(MakeDATA(test, seq_len)).transpose(0, 2, 1)
cond_test_seq = np.asarray(MakeDATA(cond_test, seq_len)).transpose(0, 2, 1)

train_seq.shape, cond_train_seq.shape, test_seq.shape, cond_test_seq.shape

((33937, 15, 48), (33937, 27, 48), (1653, 15, 48), (1653, 27, 48))

In [12]:
train_dataset = TensorDataset(torch.from_numpy(train_seq), torch.from_numpy(cond_train_seq))
train_loader = DataLoader(train_dataset, batch_size, shuffle=False)

val_dataset = TensorDataset(torch.from_numpy(val_seq), torch.from_numpy(cond_val_seq))
val_loader = DataLoader(val_dataset, batch_size, shuffle=False)

test_dataset = TensorDataset(torch.from_numpy(test_seq), torch.from_numpy(cond_test_seq))
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

In [13]:
real_data_val, real_cond_data_val = next(iter(val_loader))

# Load Model

In [14]:
# def init_weights(m):
#     if isinstance(m, nn.Linear):
#         torch.nn.init.xavier_uniform_(m.weight)
#         if m.bias is not None:
#             torch.nn.init.zeros_(m.bias)
#     elif isinstance(m, nn.LSTM):
#         for name, param in m.named_parameters():
#             if 'weight' in name:
#                 torch.nn.init.xavier_uniform_(param.data)
#             elif 'bias' in name:
#                 torch.nn.init.zeros_(param.data)

In [15]:
model = EnhancedBaseLineModel(features=train_seq.shape[1], hidden_dim=latent_dim, cond_dim=cond_train_seq.shape[1], cond_model=cond_model, device=device, channels=train_seq.shape[1])

# model.apply(init_weights)

ddpm = GaussianDiffusion1D(model, seq_length = seq_len, timesteps = timesteps, objective = objective, loss_type = 'l2', beta_schedule = beta_schedule)
ddpm = ddpm.to(device)

optim = torch.optim.Adam(ddpm.parameters(), lr = lr)
scheduler = lr_scheduler.StepLR(optim, step_size=1000, gamma=0.9)

writer = SummaryWriter(os.path.join(logging_dir, "tensorboard/"))

# Train

In [16]:
#taken from https://medium.com/@heyamit10/exponential-moving-average-ema-in-pytorch-eb8b6f1718eb
class EMA:
    def __init__(self, model, decay):
        """
        Initialize EMA class to manage exponential moving average of model parameters.
        
        Args:
            model (torch.nn.Module): The model for which EMA will track parameters.
            decay (float): Decay rate, typically a value close to 1, e.g., 0.999.
        """
        self.model = model
        self.decay = decay
        self.shadow = {}
        self.backup = {}

        # Store initial parameters
        for name, param in model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self):
        """
        Update shadow parameters with exponential decay.
        """
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def apply_shadow(self):
        """
        Apply shadow (EMA) parameters to model.
        """
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.backup[name] = param.data.clone()
                param.data = self.shadow[name]

    def restore(self):
        """
        Restore original model parameters from backup.
        """
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                param.data = self.backup[name]

In [17]:
def save_model(model_dict, ema_shadow, optimizer_dict, filepath):
    checkpoint = {
        'model_state_dict': model_dict,
        'ema_state_dict': ema_shadow,
        'optimizer_state_dict': optimizer_dict,
    }
    torch.save(checkpoint, filepath)

In [18]:
def load_checkpoint(model, ema, optimizer, filepath):
    checkpoint = torch.load(filepath)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    if 'ema_state_dict' in checkpoint:
        for name, param in model.named_parameters():
            if param.requires_grad and name in checkpoint['ema_state_dict']:
                ema.shadow[name] = checkpoint['ema_state_dict'][name].clone()
    
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [19]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.
    
    for i, (data, cond_data) in enumerate(train_loader):
        data = data.to(device)
        
        cond_data = cond_data.float()
        cond_data = cond_data.to(device)
        
        optim.zero_grad()
        
        loss = ddpm(data, cond_data)
        with torch.autograd.set_detect_anomaly(True):
            loss.backward()
        
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
        
        # ################# DEBUG ########################
        # """
        # Exploding gradients: Norms become very large (e.g., >100 or even inf).
        # Vanishing gradients: Norms shrink toward zero.
        # """
        # total_norm = 0.0
        # for p in model.parameters():
        #     if p.grad is not None:
        #         param_norm = p.grad.data.norm(2)
        #         total_norm += param_norm.item() ** 2
        # total_norm = total_norm ** 0.5
        # tb_writer.add_scalar('Gradient Norm', total_norm, epoch_index * len(train_loader) + i)
        
        # if total_norm > 1e3 or torch.isnan(torch.tensor(total_norm)):
        #     print(f"High or NaN gradient norm at step {i}: {total_norm:.4f}")
            
        # ################# DEBUG ########################
        
        optim.step()
        ema.update()
        
        running_loss += loss.item()
        
        if i % 100 == 0:
            last_loss = running_loss / 1000 # loss per batch
            tb_x = epoch_index * len(train_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.
    return last_loss

In [20]:
ema = EMA(model, decay=decay_rate)

In [21]:
epoch_number = 0
best_val_loss = 1_000_000.
save_dir =  os.path.join(logging_dir, "weights/")

ema = EMA(model, decay=decay_rate)

for epoch in tqdm(range(1, epochs+1)):
    print('epoch {}:'.format(epoch_number + 1))
    
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)
    
    running_val_loss = 0.0
    
    ema.apply_shadow()
    model.eval()
    with torch.no_grad():
        for i, (val_data, val_cond) in enumerate(val_loader):
            val_data = val_data.to(device)
        
            val_cond = val_cond.float()
            val_cond = val_cond.to(device)
            val_loss = ddpm(val_data, val_cond)
            
            running_val_loss += val_loss
    
    avg_val_loss = running_val_loss / (i + 1)
    print('Loss train {} val {}'.format(avg_loss, avg_val_loss))
    
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_val_loss },
                    epoch_number + 1)
    
    writer.flush()
    
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_dict = model.state_dict()
        best_model_opt = optim.state_dict()
        best_model_shadow = ema.shadow
        
    
    if epoch_number % save_rate == 0:
        generated_sample = ddpm.sample(batch_size, real_cond_data_val.to(device))
        generated_sample = generated_sample.cpu().numpy()
        
        plot_kde_samples(generated_sample, real_data_val,show=False, fpath=os.path.join(logging_dir, "kde/", f"kde_epoch_{epoch_number}.png"), epoch=epoch_number)
        
        model_name = f"model_epoch_{epoch_number}_val_{avg_val_loss:.4f}.pth"
        save_model(model.state_dict(), ema.shadow, optim.state_dict(), os.path.join(save_dir, model_name))
    
    ema.restore()
    epoch_number += 1

  0%|          | 0/2000 [00:00<?, ?it/s]

epoch 1:


c:\Users\Arne\anaconda3\envs\hf_diff\lib\site-packages\torch\nn\modules\rnn.py:1123: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\RNN.cpp:1410.)
  result = _VF.lstm(


Loss train 1.3463373770713807 val 1.2723420858383179


sampling loop time step: 100%|██████████| 1000/1000 [00:41<00:00, 23.95it/s]


...

sampling loop time step: 100%|██████████| 1000/1000 [00:41<00:00, 24.34it/s]


epoch 902:
Loss train 0.0015547755807638168 val 0.1505279839038849
epoch 903:
Loss train 0.0016641313079744577 val 0.15718239545822144
epoch 904:
Loss train 0.0016751339565962552 val 0.15656980872154236
epoch 905:
Loss train 0.001621507448144257 val 0.14594388008117676
epoch 906:
Loss train 0.001595003355294466 val 0.14666451513767242
epoch 907:
Loss train 0.0016635555867105722 val 0.152962788939476
epoch 908:
Loss train 0.0015704177422448992 val 0.15788963437080383
epoch 909:
Loss train 0.001634200258180499 val 0.15193381905555725
epoch 910:
Loss train 0.0015874839220196009 val 0.15949764847755432
epoch 911:
Loss train 0.001571672074496746 val 0.1528662145137787
epoch 912:
Loss train 0.0016352103427052498 val 0.15462033450603485
epoch 913:
Loss train 0.0016193594196811318 val 0.15323230624198914
epoch 914:
Loss train 0.0015919493576511741 val 0.15612585842609406
epoch 915:
Loss train 0.0015674020079895854 val 0.1542551964521408
epoch 916:
Loss train 0.001662036132067442 val 0.15779165

sampling loop time step: 100%|██████████| 1000/1000 [00:41<00:00, 24.27it/s]


epoch 1002:
Loss train 0.0015119687505066395 val 0.16629712283611298
epoch 1003:
Loss train 0.0015930710565298797 val 0.15521475672721863
epoch 1004:
Loss train 0.0015370525941252708 val 0.16002029180526733
epoch 1005:
Loss train 0.0014674914488568903 val 0.1596078723669052
epoch 1006:
Loss train 0.0015139001673087477 val 0.16359573602676392
epoch 1007:
Loss train 0.0014771541142836212 val 0.16047263145446777
epoch 1008:
Loss train 0.0015784609531983734 val 0.1667172610759735
epoch 1009:
Loss train 0.0015269925305619837 val 0.15235790610313416
epoch 1010:
Loss train 0.0015519520444795488 val 0.1600360870361328
epoch 1011:
Loss train 0.001533260243013501 val 0.154218852519989
epoch 1012:
Loss train 0.0015023225713521242 val 0.1583917886018753
epoch 1013:
Loss train 0.0015292229345068335 val 0.15360954403877258
epoch 1014:


KeyboardInterrupt: 

In [ ]:
save_model(best_model_dict, best_model_shadow, best_model_opt, os.path.join(logging_dir, "weights/", "best_model.pth"))

In [ ]:
load_checkpoint(model, ema, optim, os.path.join(logging_dir, "weights/", "best_model.pth"))

In [ ]:
paths = []
for kde_plot in os.listdir(os.path.join(logging_dir, "kde/")):
    paths.append(os.path.join(logging_dir, "kde/", kde_plot))

make_gif_from_images(paths, os.path.join(logging_dir, "kde/", "kde_progression.gif"))

# Evaluation

In [ ]:
real_data_test, real_cond_data_test = next(iter(test_loader))

In [ ]:
with torch.no_grad():
    samples = ddpm.sample(batch_size, real_cond_data_test.to(device))
    samples = samples.cpu().numpy()

print(f"Samples shape: {samples.shape}")

In [ ]:
vizual_comparison(samples, real_data_test, os.path.join(logging_dir, "viz/", "pca_umap_tsne_all_batches.png"), use_all_data=True);

In [ ]:
vizual_comparison(samples, real_data_test, os.path.join(logging_dir, "viz/", "pca_umap_tsne_per_batch.png"));

In [ ]:
mmd_histogram_per_customer(samples, real_data_test, fpath=os.path.join(logging_dir, "mmd/", "mmd.png"))

In [ ]:
plot_jsd_per_customer(samples, real_data_test, os.path.join(logging_dir, "jsd/", "jsd.png"))

In [ ]:
plot_kde_samples(samples, real_data_test, fpath=os.path.join(logging_dir, "kde/", f"kde.png"))

In [ ]:
batch_idx = np.random.randint(0, 256) 
customer_indices = np.random.randint(0, 15, size=15)
fig, axs = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

with torch.no_grad():
    sample = ddpm.sample(1, real_cond_data_test.to(device)) 
    sample = sample.cpu().numpy() 

for i in customer_indices:
    axs[0].plot(sample[0, i], alpha=0.7, label=f'Customer {i}')
    axs[1].plot(real_data_test[0, i], alpha=0.7, label=f'Customer {i}')

axs[0].set_title("Generated Data")
axs[1].set_title("Real Data")

for ax in axs:
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')
    ax.grid(True)

plt.tight_layout()
plt.savefig(os.path.join(logging_dir, "ts_sample/", "samples.png"))
plt.show()